# 10. Prepare data for cell-entity annotation taks
Prepare the json of the each page to perform entity linking. 

Include :
* idem and ditto replacement (for pages created for annotations)
* named entity recognition on cells who might require it
* plot numbers post-treatement [UNDER DEVELOPPEMENT]

In [1]:
import pandas as pd
import glob
import json

In [2]:
import sys
import os
from pathlib import Path
# Access to the utils directory
current_dir = os.getcwd()
utils_dir = os.path.join(current_dir, '..', 'utils')
sys.path.append(utils_dir)

In [3]:
from string_utils import TableValuesPostTreatment, NormalizeText

[nltk_data] Downloading package punkt to /home/STual/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/STual/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
ROOT = "/home/STual/DAN-cadastre/inference/LHAY/"
JSONS = glob.glob(ROOT + '*.json')
print("Number of pages : " + str(len(JSONS)))

Number of pages : 26


## 1. Named Entity Recognition (NER)
This step aims to structure more precisly the content of a cell. In this example, its the taxpayers cells how are treated by a fine-tuned model.

In [5]:
BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve()
OUT_BASE = Path('/home/STual/DAN-cadastre/outputs/NER_inference').resolve()

### Model to run
MODEL_PATH = OUT_BASE / 'tmp'

FINETUNED_MODEL = MODEL_PATH / "ner-joint-labelling-io-1534/checkpoint-900"
FINETUNED_MODEL

PosixPath('/home/STual/DAN-cadastre/outputs/NER_inference/tmp/ner-joint-labelling-io-1534/checkpoint-900')

### 1.1 Load NER model

In [6]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline

LIMIT = 100

tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL) #Param : tokenizer du modele souhaité
model = AutoModelForTokenClassification.from_pretrained(FINETUNED_MODEL) #Modèle choisi

#Classification des entités
nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple", use_fast=True, ignore_labels=["O+O"])

/home/STual/.venv/venv_sti/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


### 1.2 Retrieve distinct mentions in a column

In [7]:
CELL_ENTITY_TOKEN = "Ⓒ" #Token of the column to treat : here it's the taxpayers

In [8]:
import time
import datetime

column_values = []
for JSON in JSONS:
    with open(JSON,'r',encoding='utf8') as f:
        page = json.load(f)
    page_uuid = JSON.replace(ROOT,"").replace('.json','')
    
    #Retrieve all the taxpayers text of the page
    column_values_by_page = []
    counter = 0
    for line in page["entities"]:
        keys = line.keys()
        if CELL_ENTITY_TOKEN in keys:
            column_values_by_page.append([counter, line[CELL_ENTITY_TOKEN]])
        else:
            column_values_by_page.append([counter, "MISSING"])
        counter += 1
    column_values.append([page_uuid, column_values_by_page])
#This return a list [page_uuid,line,list_od_taxpayers]. Idem and Ditto should have been replaced

This return a list [page_uuid, line, list_od_taxpayers].

In [9]:
#List the distinct taxpayers values 
start = time.time()
distinct_cell_values = []
for page in column_values:
    for line in page[1]:
        try:
            if line[1]["interpreted_text"] not in distinct_cell_values:
                distinct_cell_values.append(line[1]["interpreted_text"])
            else:
                continue
        except:
            print(line)

[11, 'MISSING']


In [10]:
distinct_cell_values

['Godefroy denis V↑e↓',
 'Gabillot claude',
 'mul',
 'Nul',
 'Benoist (lanoslas',
 'Chevalier louis lt↑re↓',
 'Massuet denis marie',
 'Serre de Saint roman',
 'Chanevas louis',
 'Sevin jean achille',
 'Jacquet Jean ch↑es↓',
 'Lebourtier à Fresnes',
 'peuvier Et↑e↓',
 'Joseph micolast e',
 'Roret jacques michel',
 'Desaintpaul Bonisanc→V↑e↓',
 'Leclerc claude félix',
 'Chevreul michel',
 'Baschoux denis',
 'Demay marie f↑ois↓',
 'Delaunay J↑n↓ b↑te↓',
 'Privière antoine germain',
 'Vincent f↑ois↓ mathury',
 'Brice J↑n↓ fils à la→Branve f↑re↓',
 'Goyard michel V↑e↓',
 'Thibault marie J↑n↓',
 'Mateu j↑ques↓ p↑re↓',
 'Privière antoine',
 'Piviere andré louis',
 'Bronzac p↑re↓ marie',
 'Maucait antoine',
 'Michaux Carrier',
 'Deu f↑ois↓ laurent',
 'Micolas antoine',
 'Deschamps J↑n↓ b↑te↓',
 'Trottier J↑ques↓ J↑n↓',
 'Boncorps marie antoine',
 'hillet a chevilly',
 'T horin antoine',
 'Godefroy claude joseph',
 'David Etie ne philippe',
 'Lampy à Versaillat',
 'Mamtzay marie geune',
 'Cusze

### 1.3 Detect crossed-out text and perform NER
* Detect crossed out text and separate content of the cell in two parts : crossed-out and non-crossed out text
* Proceed to named entities recogniion in both crossed-out and non crossed out text

In [11]:
ner_annotated_cells = {}
ner_annotated_cells_crossed_out = {}

start = time.time()
for t in distinct_cell_values :
    text, crossed_out = NormalizeText.separate_crossed_out(t)
    
    # Not crossed out
    res = nlp(text)
    for elem in res:
        elem["entity_group"] = elem["entity_group"].replace('+O','')
        elem["score"] = round(float(elem["score"]),4)
    ner_annotated_cells.update({t:res})
    
    #Crossed out
    if len(crossed_out) > 0:
        res2 = nlp(crossed_out)
        for elem in res2:
            elem["entity_group"] = elem["entity_group"].replace('+O','')
            elem["score"] = round(float(elem["score"]),4)
        ner_annotated_cells_crossed_out.update({t:res2})
        
end = time.time()
run = end-start
average_runtime = run/len(ner_annotated_cells)
print(f"Running time for {len(ner_annotated_cells)} entries is {round(run,3)} seconds (average per entry : {round(average_runtime*1000,3)} milliseconds)")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Running time for 289 entries is 2.394 seconds (average per entry : 8.284 milliseconds)


### 1.4 Save the results into the JSONS

In [12]:
import time
import datetime
from collections import defaultdict

column_values = []
default_ner_properties = ["name","firstnames","activity","address","title","birthname","familystatus"]

for JSON in JSONS:
    with open(JSON) as f:
        page = json.load(f)
    page_uuid = JSON.replace(ROOT,"").replace('.json','')
    
    for line in page["entities"]:
        if CELL_ENTITY_TOKEN in line.keys():
            line[CELL_ENTITY_TOKEN]["ner_interpreted_text"] = ner_annotated_cells[line[CELL_ENTITY_TOKEN]["interpreted_text"]]
            # Group by entity_group and concatenate words
            data = line[CELL_ENTITY_TOKEN]["ner_interpreted_text"]
            grouped = defaultdict(str)
            grouped["name"]=""
            grouped["firstnames"]=""
            grouped["birthname"]=""
            grouped["address"]=[]
            grouped["familystatus"]=[]
            grouped["activity"]=[]
            grouped["title"]=[]
            for item in data:
                if item['entity_group'] in ("name","firstnames","birthname"):
                    grouped[item['entity_group']] += item['word'] + ' '
                else:
                    grouped[item['entity_group']].append(item['word'])
            
            retrieved_entities = grouped.keys()
            for ne in default_ner_properties:
                if ne not in retrieved_entities:
                   grouped[ne] = ""
    
            line[CELL_ENTITY_TOKEN]["ner_interpreted_text_grouped"] = grouped
            
            if line[CELL_ENTITY_TOKEN]["interpreted_text"] in ner_annotated_cells_crossed_out.keys():
                if len(ner_annotated_cells_crossed_out[line[CELL_ENTITY_TOKEN]["interpreted_text"]]) > 0:
                    line[CELL_ENTITY_TOKEN]["ner_interpreted_text_crossed_out"] = ner_annotated_cells_crossed_out[line[CELL_ENTITY_TOKEN]["interpreted_text"]]
    
    with open(JSON,'w', encoding='utf-8') as f:
        json.dump(page, f, ensure_ascii=False, indent=4)

## 2. Item and Dittos propagations among named entities

## 3. Numbers post-correction

!!!! NOT WORKING !!!!

In [13]:
def detect_anomalies_and_propose_corrections(plots):
    """
    Detect anomalies in the sequence of plot numbers and propose corrections.
    The plots list contains numbers or text errors.
    
    :param plots: List of plot numbers (or text) from the table.
    :return: List of anomalies and proposed corrections.
    """
    
    # Helper function to check if a number is an integer
    def is_integer(value):
        try:
            int(value)
            return True
        except ValueError:
            return False
    
    # Convert all values to integers or NaN if they are invalid
    plot_numbers = [int(plot) if is_integer(plot) else None for plot in plots]

    # Detect anomalies
    anomalies = []
    corrected_plots = plot_numbers.copy()

    for i in range(1, len(plot_numbers)):
        current_plot = plot_numbers[i]
        previous_plot = plot_numbers[i-1]
        biggest = previous_plot
        
        # Case 1: Plot is NaN or not an integer
        if current_plot is None:
            anomalies.append(f"Anomaly: Invalid value at position {i} (Non-integer or empty).")
            continue
        
        # Case 2: Plot is out of order (non-increasing or skipped numbers)
        if current_plot < previous_plot:
            anomalies.append(f"Anomaly: Out of order at position {i} ({current_plot}). Expected value greater than {previous_plot}.")
            # Propose correction: suggest the number should be the next in sequence
            corrected_plots[i] = previous_plot + 1

        # Case 3: A number is repeated in non-sequential lines
        if current_plot == previous_plot and i > 1 and plot_numbers[i-2] != current_plot:
            anomalies.append(f"Anomaly: Repeated number {current_plot} at position {i}, non-sequential.")
            # Propose correction: suggest the number should be the next in sequence
            corrected_plots[i] = previous_plot
            
    # Propose minimal corrections for anomalies detected
    corrections = []
    for i, anomaly in enumerate(anomalies):
        # Simply add the corrected plot number
        corrections.append({
            'anomaly': anomaly,
            'corrected_value': corrected_plots[i]
        })
    
    # Return anomalies and corrected values
    return corrections, corrected_plots

def separate_digits_and_text(input_string):
    digits = ''
    text = ''
    
    for char in input_string:
        if char.isdigit():
            digits += char
        else:
            text += char
    return digits, text

In [14]:
# Test with the provided sequence
plots_example = ["71", "73", "73", "73", "75", "75", "76", "78", "59", "50", "81", "52"]
anomalies, corrected_plots = detect_anomalies_and_propose_corrections(plots_example)
anomalies, corrected_plots

([{'anomaly': 'Anomaly: Repeated number 73 at position 2, non-sequential.',
   'corrected_value': 71},
  {'anomaly': 'Anomaly: Repeated number 75 at position 5, non-sequential.',
   'corrected_value': 73},
  {'anomaly': 'Anomaly: Out of order at position 8 (59). Expected value greater than 78.',
   'corrected_value': 73},
  {'anomaly': 'Anomaly: Out of order at position 9 (50). Expected value greater than 59.',
   'corrected_value': 73},
  {'anomaly': 'Anomaly: Out of order at position 11 (52). Expected value greater than 81.',
   'corrected_value': 75}],
 [71, 73, 73, 73, 75, 75, 76, 78, 79, 60, 81, 82])

In [15]:
anomalies, corrected_plots = detect_anomalies_and_propose_corrections(corrected_plots)

In [16]:
corrected_plots

[71, 73, 73, 73, 75, 75, 76, 78, 79, 80, 81, 82]

In [20]:
import time
import datetime
from collections import defaultdict

column_values = []
CELL_ENTITY_TOKEN = "Ⓕ"
remove_chars_regex = '→()↑↓×.,!?;:-@#$%^&* '
replacement_char = ''

for JSON in JSONS:
    print(JSON)
    with open(JSON) as f:
        page = json.load(f)
    page_uuid = JSON.replace(ROOT,"").replace('.json','')

    numbers, texts = [], []
    for line in page["entities"]:
        keys = line.keys()
        if CELL_ENTITY_TOKEN in keys:
            line[CELL_ENTITY_TOKEN]["postcorrected_text"] = ''
            if "interpreted_text" in list(line[CELL_ENTITY_TOKEN].keys()):
                num, text = separate_digits_and_text(line[CELL_ENTITY_TOKEN]["interpreted_text"])
                numbers.append(num)
                texts.append(text)
            else:
                line[CELL_ENTITY_TOKEN]["interpreted_text"] = ""
        else:
            numbers.append(0)
            texts.append('MISSING')
    anomalies, corrected_plots = detect_anomalies_and_propose_corrections(numbers)
    anomalies, corrected_plots = detect_anomalies_and_propose_corrections(corrected_plots)

    
    for i in range(len(page["entities"])):
        if CELL_ENTITY_TOKEN in page["entities"][i].keys():
            res = str(corrected_plots[i]) + texts[i]
            res = NormalizeText.replace_characters(res, remove_chars_regex, replacement_char)
            if numbers[i] != 10000000:
                page["entities"][i][CELL_ENTITY_TOKEN]["postcorrected_text"] = res
            else:
                page["entities"][i][CELL_ENTITY_TOKEN]["postcorrected_text"] = page["entities"][i][CELL_ENTITY_TOKEN]["interpreted_text"]
    
    with open(JSON,'w', encoding='utf-8') as f:
        json.dump(page, f, ensure_ascii=False, indent=4)

/home/STual/DAN-cadastre/inference/LHAY/3435dedc-ddc0-4dfd-b834-d04148e44ddf.json
/home/STual/DAN-cadastre/inference/LHAY/79ee0e05-59c0-4770-a20e-326574627577.json
/home/STual/DAN-cadastre/inference/LHAY/683e7c3e-e537-44ba-8a02-3c4678d86b2f.json
/home/STual/DAN-cadastre/inference/LHAY/7cc717f5-4612-45f0-8181-7e341e937880.json
/home/STual/DAN-cadastre/inference/LHAY/9e52f7f9-0e66-4753-bcf3-d3cb0916678c.json


IndexError: list index out of range

## 4. URIS

In [6]:
import uuid

ADD_LINE_URIS = False

if ADD_LINE_URIS:
    for JSON in JSONS:
        print(JSON)
        with open(JSON) as f:
            page = json.load(f)
        page_uuid = JSON.replace(ROOT,"").replace('.json','')
    
        numbers, texts = [], []
        for line in page["entities"]:
            line["uri"] = str(uuid.uuid4())
        
        with open(JSON,'w', encoding='utf-8') as f:
            json.dump(page, f, ensure_ascii=False, indent=4)

/home/STual/DAN-cadastre/inference/LHAY/3ab13de2-f0b2-436f-9aa0-3b040e79975b.json
/home/STual/DAN-cadastre/inference/LHAY/6b08bf95-e3f5-423c-afa2-b088cc0ca11a.json
/home/STual/DAN-cadastre/inference/LHAY/3435dedc-ddc0-4dfd-b834-d04148e44ddf.json
/home/STual/DAN-cadastre/inference/LHAY/79ee0e05-59c0-4770-a20e-326574627577.json
/home/STual/DAN-cadastre/inference/LHAY/bd3d83f2-e9d7-4311-a9eb-c528f4501f27.json
/home/STual/DAN-cadastre/inference/LHAY/683e7c3e-e537-44ba-8a02-3c4678d86b2f.json
/home/STual/DAN-cadastre/inference/LHAY/7cc717f5-4612-45f0-8181-7e341e937880.json
/home/STual/DAN-cadastre/inference/LHAY/9e52f7f9-0e66-4753-bcf3-d3cb0916678c.json
/home/STual/DAN-cadastre/inference/LHAY/2c59c9c6-5ed5-4fc1-85cd-4d162c324239.json
/home/STual/DAN-cadastre/inference/LHAY/8d60076b-985d-4374-acb8-c8f17db50d76.json
/home/STual/DAN-cadastre/inference/LHAY/f10c1378-c307-4b24-95f8-1c016610d4af.json
/home/STual/DAN-cadastre/inference/LHAY/92fb30f9-6891-46e0-adb0-ad396a3ad2dc.json
/home/STual/DAN-